**Корректность проверена на Python 3.7:**
+ pandas 0.23.0
+ numpy 1.14.5
+ scipy 1.1.0

# Множественная проверка гипотез

In [75]:
import pandas as pd
import numpy as np

from scipy.stats import wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

In [31]:
import scipy
print(np.__version__)
print(pd.__version__)
print(scipy.__version__)

1.16.2
0.24.2
1.2.1


Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

AUCs.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.
Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [43]:
data = pd.read_csv('AUCs.txt', sep = '\t', header = 0, parse_dates = [2])

In [44]:
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [63]:
data.dtypes

Unnamed: 0     object
C4.5          float64
C4.5+m         object
C4.5+cf       float64
C4.5+m+cf     float64
dtype: object

In [64]:
data['C4.5+m'] = data[['C4.5+m']].astype('float')

In [65]:
data.dtypes

Unnamed: 0     object
C4.5          float64
C4.5+m        float64
C4.5+cf       float64
C4.5+m+cf     float64
dtype: object

In [66]:
classificators_list = data.columns[1:]
classificators_pairs = [[i, j] for i in classificators_list 
                        for j in classificators_list if i < j]
classificators_pairs

[['C4.5', 'C4.5+m'],
 ['C4.5', 'C4.5+cf'],
 ['C4.5', 'C4.5+m+cf'],
 ['C4.5+m', 'C4.5+m+cf'],
 ['C4.5+cf', 'C4.5+m'],
 ['C4.5+cf', 'C4.5+m+cf']]

In [87]:
max_difference = 10
j = 0
n = 0
stats_list = []
pv_list = []
for i in range(len(classificators_pairs)):
    stat, p_value = wilcoxon(data[classificators_pairs[i][0]], data[classificators_pairs[i][1]])
    print classificators_pairs[i], stat, p_value
    stats_list.append(stat)
    pv_list.append(p_value)
    if max_difference > stat:
        max_difference = stat
        j = i
    if p_value <= 0.05:
        n += 1

print 'max significance difference is for ', classificators_pairs[j], max_difference
print 'p-palue <= 0.05: ', n

['C4.5', 'C4.5+m'] 6.0 0.009632975772588674
['C4.5', 'C4.5+cf'] 43.0 0.861262330095348
['C4.5', 'C4.5+m+cf'] 11.0 0.015906444101703374
['C4.5+m', 'C4.5+m+cf'] 33.5 0.40153441947093693
['C4.5+cf', 'C4.5+m'] 17.5 0.05030093442929343
['C4.5+cf', 'C4.5+m+cf'] 10.0 0.022909099354356588
max significance difference is for  ['C4.5', 'C4.5+m'] 6.0
p-palue <= 0.05:  3


Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [82]:
reject_holm, p_corrected, a1, a2 = multipletests(stats_list, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [83]:
reject_holm

array([False, False, False, False, False, False])

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [86]:
reject_bh, p_corrected, a1, a2 = multipletests(pv_list, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
reject_bh

array([ True, False,  True, False, False,  True])